In [1]:
from bs4 import BeautifulSoup
import glob
import os

courses_directory = 'nbgrader'
#course = ""
#assignment = ""

#feedback = os.path.join(courses_directory, course, 'feedback', '*', assignment, '*.html')
#feedback
result = ""

def getResult():
    global result
    return result

def setPath(c, s):
    course = c
    sid = s
    feedback = os.path.join(courses_directory, course, 'feedback', s, '*', '*.html')
    return feedback
    #print(feedback)

def fetch(feedback):
    global result
    feedback_files = glob.glob(feedback)
    #feedback_files
    result = ""
    if (not feedback_files):
        result = "no feedback to summarize."
        return result
    
    for f in feedback_files:
        
        normalized_path = os.path.normpath(f)
        print(normalized_path)
        # Split the normalized path into its components
        parts = normalized_path.split(os.sep)  # os.sep is the appropriate separator
        index = parts.index('feedback')
        assignmentName = parts[index + 3]
        
        with open(normalized_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        soup = BeautifulSoup(html_content, 'html.parser')
        
        ph = soup.find('div', class_="panel-heading")
        overall = str(ph.find('h4')).split("Score:")[1].split(")")[0].strip()
        result += (assignmentName + "\n")
        result += (f"Overall Score: {overall}\n")
    
        qs = ph.find('ol').find_all('li')
        count = 1
        for q in qs:
            href = str(q)[str(q).find('#')+1:str(q).find('">')]
            t = BeautifulSoup(str(q), 'html.parser').get_text()
            
            if (t == "Comment"):
                anchor = soup.find('a', attrs={'name': href})
                div = anchor.find_parent('div')
                comment = BeautifulSoup(str(div.find('div', class_="panel-footer").find('div').get_text()), 'html.parser')
                result += (comment.get_text().strip() + "\n")
            else:
                result += (str(count) + ". " + t + "\n")
                count += 1   
            
        result += ("--------------------------\n")
    return result

In [2]:
import ipywidgets as widgets
from IPython.display import display
import os
import threading
import time

op = None
th = 0

course = widgets.Dropdown(
    options=[],
    description='Course:',
    disabled=False
)

student = widgets.Text(
    description='Student ID:',
    disabled=False
)

button = widgets.Button(
    description='Submit',
    layout=widgets.Layout(margin='0px 0px 0px 90px'), 
    disabled=True
)

textbox = widgets.Textarea(
    description='Output:',
    layout=widgets.Layout(width='50%', height='300px'),  # Set width and height
    disabled=True
)

pathText = widgets.Text(
    description="Current Path:",
    value=courses_directory,
    disabled=True
)

pathUpdate = widgets.Text(
    description="Update Path:",
)

btnUpdate = widgets.Button(
    description='Update',
    layout=widgets.Layout(margin='0px 0px 0px 90px'),
)

title = widgets.HTML(
    value="<center><h1>AI Summarizing</h1></center>"
)

grid = widgets.GridspecLayout(3, 2, width='50%')
grid[0,0] = course
grid[1,0] = student
grid[2,0] = button
grid[0,1] = pathText
grid[1,1] = pathUpdate
grid[2,1] = btnUpdate

box_layout = widgets.Layout(display='flex',
                    flex_flow='column', 
                    align_items='center',
                    justify_content='center',
                    width='100%',
                    height='100%',
                    margin='250px 0px 0px 0px')

box = widgets.Box(children=[title, grid, textbox], layout=box_layout)

def course_initilize():
    courses = [name for name in os.listdir(courses_directory) if os.path.isdir(os.path.join(courses_directory, name))]
    courses.insert(0, "---SELECT COURSE---")
    course.options = courses
    course.value = course.options[0]

def countTime():
    global op
    global th
    i = 0
    th = 0
    dot = [".", "..", "..."]
    while (op is None):
        textbox.value = "Executing" + dot[i%3] + "\n" + str(i) + " seconds"
        time.sleep(1)
        i += 1
    th = i

def student_change(change):
    if student.value != "" and course.value != "---SELECT COURSE---":
        button.disabled = False
    else:
        button.disabled = True

def on_button_clicked(b):
    global op
    global th
    f = setPath(course.value, student.value)
    t = fetch(f)
    if (t == "no feedback to summarize."):
        textbox.value = t
    elif (t):
        textbox.value = "Executing..."
        button.disabled = True
        op = None
        threading.Thread(target=countTime).start()
        %run -i summarize_student.ipynb
        op = summarize(t)
        time.sleep(1)
        textbox.value = op + "\nExecuted for " + str(th) + " seconds"
        button.disabled = False

def on_btnUpdate_clicked(b):
    global courses_directory
    if os.path.isdir(pathUpdate.value):
        courses_directory = pathUpdate.value
        pathText.value = courses_directory
        course_initilize()
        if textbox.value == "Invalid path":
            textbox.value = ""
    else:
        textbox.value = "Invalid path"

course_initilize()
student.observe(student_change, names='value')
button.on_click(on_button_clicked)
btnUpdate.on_click(on_btnUpdate_clicked)

display(box)

Box(children=(HTML(value='<center><h1>AI Summarizing</h1></center>'), GridspecLayout(children=(Dropdown(descri…

In [5]:
#test
t = """
Card.html
Overall Score: 4.0 / 4.0
1. Written response (Score: 1.0 / 1.0)
Comments: Your code is generally well-structured. Consider using more descriptive variable names to improve readability.
2. Written response (Score: 1.0 / 1.0)
Comments: Great job on including comments! They help clarify your thought process. However, ensure comments are concise and relevant.
3. Written response (Score: 1.0 / 1.0)
Comments: The code is mostly well-formatted. Consistent indentation and spacing enhance readability.
4. Written response (Score: 1.0 / 1.0)
Comments: The program runs as expected and produces the correct output for the provided test cases. Well done!
--------------------------
Improved_Quadratic_Equation_Solver.html
Overall Score: 8.7 / 10.0
1. Test cell (Score: 0.8 / 1.0)
Comments: Consider testing edge cases to ensure your code handles all possible inputs robustly.
2. Test cell (Score: 0.8 / 1.0)
Comments: The algorithm is efficient for the given problem. However, you might explore alternative approaches that could reduce time complexity.
3. Test cell (Score: 0.8 / 1.0)
Comments: Be mindful of memory usage. For large datasets, consider using more efficient data structures.
4. Test cell (Score: 0.8 / 1.0)
Comments: You've repeated some code in multiple places. Try to refactor it into functions to adhere to the "Don't Repeat Yourself" principle.
5. Written response (Score: 1.0 / 1.0)
Comments: Your code is generally well-structured. Consider using more descriptive variable names to improve readability.
6. Written response (Score: 0.5 / 1.0)
Comments: Explore ways to optimize your code further, especially if the input size increases significantly.
7. Test cell (Score: 1.0 / 1.0)
Comments: Implement error handling to manage unexpected inputs gracefully.
8. Test cell (Score: 1.0 / 1.0)
Comments: Your code is generally well-structured. Consider using more descriptive variable names to improve readability.
9. Test cell (Score: 1.0 / 1.0)
Comments: The code is mostly well-formatted. Consistent indentation and spacing enhance readability.
10. Test cell (Score: 1.0 / 1.0)
Comments: The program runs as expected and produces the correct output for the provided test cases. Well done!
--------------------------
Parsing_and_Validation.html
Overall Score: 3.3 / 4.0
1. Test cell (Score: 0.8 / 1.0)
Comments: You've repeated some code in multiple places. Try to refactor it into functions to adhere to the "Don't Repeat Yourself" principle.
2. Test cell (Score: 1.0 / 1.0)
Comments: The code is mostly well-formatted. Consistent indentation and spacing enhance readability.
3. Test cell (Score: 0.5 / 1.0)
Comments: Implement error handling to manage unexpected inputs gracefully.
4. Test cell (Score: 1.0 / 1.0)
Comments: Your code is generally well-structured. Consider using more descriptive variable names to improve readability.
--------------------------
"""

%run -i summarize_student.ipynb
op = summarize(t)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


CPU times: total: 0 ns
Wall time: 0 ns
Content length: 2886 chars, 856 tokens.
Content sample:

Card.html
Overall Score: 4.0 / 4.0
1. Written response (Score: 1.0 / 1.0)
Comments: Your code is generally well-structured. Consider using more descriptive variable names to improve readability.
2. W


Using summarizer: base


> Entering new LLMChain chain...
Prompt after formatting:

The following text delimited by tripple backquotes is about the performance of a student on assignments in a course. Each assignment is separated by a line of dashes (----).
Each assignment's data includes:
1. The assignment name
2. The overall score for the assignment
3. A series of questions, including scores and comments for each question.
Score: X / Y means X marks gained in a Y marks question. A question is answered correctly if X = Y. 

Write a summary of it.
Return your response as report which covers the statistic of the student in the course. Highlight the strength and weakness of the student. Do not m